In [10]:
###getting the reports local url's:###
##extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            else:
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()


##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

In [26]:
#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2025), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths
download_all_chapters()

⬇️ Download 100% for 01.docx
✅ Saved 01.docx to /content/reports/2001/01_01.docx
⬇️ Download 100% for 02.docx
✅ Saved 02.docx to /content/reports/2001/02_02.docx
⬇️ Download 100% for 03.docx
✅ Saved 03.docx to /content/reports/2001/03_03.docx
⬇️ Download 100% for 04.docx
✅ Saved 04.docx to /content/reports/2001/04_04.docx
⬇️ Download 100% for 05.docx
✅ Saved 05.docx to /content/reports/2001/05_05.docx
⬇️ Download 100% for 06.docx
✅ Saved 06.docx to /content/reports/2001/06_06.docx
⬇️ Download 100% for 07.docx
✅ Saved 07.docx to /content/reports/2001/07_07.docx
⬇️ Download 100% for 08.docx
✅ Saved 08.docx to /content/reports/2001/08_08.docx
⬇️ Download 100% for 09.docx
✅ Saved 09.docx to /content/reports/2001/09_09.docx
⬇️ Download 100% for 10.docx
✅ Saved 10.docx to /content/reports/2001/10_10.docx
⬇️ Download 100% for 11.docx
✅ Saved 11.docx to /content/reports/2001/11_11.docx
⬇️ Download 100% for 12.docx
✅ Saved 12.docx to /content/reports/2001/12_12.docx
⬇️ Download 100% for 13.docx

{'2001': {1: '/content/reports/2001/01_01.docx',
  2: '/content/reports/2001/02_02.docx',
  3: '/content/reports/2001/03_03.docx',
  4: '/content/reports/2001/04_04.docx',
  5: '/content/reports/2001/05_05.docx',
  6: '/content/reports/2001/06_06.docx',
  7: '/content/reports/2001/07_07.docx',
  8: '/content/reports/2001/08_08.docx',
  9: '/content/reports/2001/09_09.docx',
  10: '/content/reports/2001/10_10.docx',
  11: '/content/reports/2001/11_11.docx',
  12: '/content/reports/2001/12_12.docx',
  13: '/content/reports/2001/13_13.docx',
  14: '/content/reports/2001/14_14.docx',
  15: '/content/reports/2001/15_15.docx'},
 '2002': {1: '/content/reports/2002/01_01.docx',
  2: '/content/reports/2002/02_02.docx',
  3: '/content/reports/2002/03_03.docx',
  4: '/content/reports/2002/04_04.docx',
  5: '/content/reports/2002/05_05.docx',
  6: '/content/reports/2002/06_06.docx',
  7: '/content/reports/2002/07_07.docx',
  8: '/content/reports/2002/08_08.docx',
  9: '/content/reports/2002/09_09.

In [27]:
import os
import subprocess
from docx import Document

def convert_doc_to_docx(doc_path):
    """
    Convert a .doc file to .docx using LibreOffice (headless mode)
    Returns the path to the converted .docx file
    """
    if not doc_path.endswith(".doc"):
        return doc_path  # already .docx

    converted_path = doc_path + "x"  # simple .docx path
    os.makedirs(os.path.dirname(converted_path), exist_ok=True)

    # LibreOffice command-line conversion
    subprocess.run([
        "libreoffice",
        "--headless",
        "--convert-to",
        "docx",
        "--outdir",
        os.path.dirname(doc_path),
        doc_path
    ], check=True)

    return converted_path

def convert_wmf_to_png(wmf_path, png_path):
    """Convert .wmf or .x-wmf to .png using ImageMagick"""
    subprocess.run([
        "convert", wmf_path, png_path
    ], check=True)

def extract_images_from_docx(docx_path, output_dir):
    """
    Extracts all images from a .docx file and saves them in output_dir.
    WMF images are automatically converted to PNG.
    """
    doc = Document(docx_path)
    image_count = 0
    for i, rel in enumerate(doc.part._rels):
        rel = doc.part._rels[rel]
        if "image" in rel.target_ref:
            image_count += 1
            img_data = rel.target_part.blob
            ext = rel.target_part.content_type.split('/')[-1]
            if ext == 'jpeg':
                ext = 'jpg'

            img_name = f"image_{i+1}.{ext}"
            img_path = os.path.join(output_dir, img_name)
            os.makedirs(output_dir, exist_ok=True)
            with open(img_path, "wb") as f:
                f.write(img_data)

            # Convert WMF to PNG
            if ext in ["x-wmf", "wmf"]:
                png_path = os.path.splitext(img_path)[0] + ".png"
                try:
                    convert_wmf_to_png(img_path, png_path)
                    os.remove(img_path)  # remove original WMF
                    print(f"🔄 Converted {img_path} → {png_path}")
                except Exception as e:
                    print(f"❌ Failed to convert {img_path}: {e}")

    return image_count

# Make sure LibreOffice & ImageMagick are installed
!apt-get update
!apt-get install -y libreoffice imagemagick

# Paths
reports_dir = "/content/reports"          # local folder with reports
images_output_dir = "/content/reports_images"

years = [str(y) for y in range(2001, 2025)]
chapters = list(range(1, 16))  # chapters 1-15

for year in years:
    for chapter in chapters:
        chapter_str = f"{chapter:02}"
        year_folder = os.path.join(reports_dir, year)
        if not os.path.exists(year_folder):
            continue

        # Find all .doc and .docx files for this chapter
        chapter_files = [f for f in os.listdir(year_folder)
                         if f.startswith(chapter_str) and (f.endswith(".docx") or f.endswith(".doc"))]

        if not chapter_files:
            print(f"⚠️ No Word files found for year {year}, chapter {chapter_str}")
            continue

        for file_name in chapter_files:
            file_path = os.path.join(year_folder, file_name)
            # Convert .doc to .docx if needed
            if file_path.endswith(".doc"):
                print(f"🔄 Converting {file_path} to .docx")
                try:
                    file_path = convert_doc_to_docx(file_path)
                except Exception as e:
                    print(f"❌ Conversion failed for {file_path}: {e}")
                    continue

            out_dir = os.path.join(images_output_dir, year, chapter_str)
            count = extract_images_from_docx(file_path, out_dir)
            print(f"🖼 Extracted {count} images from {file_path} to {out_dir}")


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.4 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,002 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [43.0 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:13 http://archive.

KeyboardInterrupt: 

In [ ]:
import os
import json
import pandas as pd
from docx import Document
!apt-get install -y libreoffice
import subprocess

def convert_doc_to_docx(base_dir="/content/reports"):
    for root, _, files in os.walk(base_dir):
        for fname in files:
            if fname.endswith(".doc") and not fname.endswith(".docx"):
                fpath = os.path.join(root, fname)
                print(fpath)
                subprocess.run([
                    "libreoffice", "--headless", "--convert-to", "docx", fpath, "--outdir", root
                ])

# convert all docs
convert_doc_to_docx("/content/reports")

In [18]:
!pip install olefile
!pip install docx2txt

import docx2txt as d2t
import os
import json
from docx import Document
import olefile  # For OLE object detection (Excel embedded)

base_dir = "/content/reports"
out_dir = "/content/graphs"
os.makedirs(out_dir, exist_ok=True)

graph_summary = {}  # identifier -> Hebrew name or None

def is_excel_ole(ole_data):
    """Check if OLE object is an Excel file"""
    try:
        ole = olefile.OleFileIO(ole_data)
        return ole.exists('Workbook') or ole.exists('Book')
    except:
        return False

def extract_graphs_from_reports(base_dir=base_dir, out_dir=out_dir):
    for year in range(2001, 2002):
        print(f"📂 Year {year}")
        year_path = os.path.join(base_dir, str(year))
        if not os.path.isdir(year_path):
            continue

        for fname in os.listdir(year_path):
            if not fname.endswith(".docx"):
                continue

            chapter = fname.split("_")[0]
            fpath = os.path.join(year_path, fname)

            try:
                doc = Document(fpath)
            except Exception as e:
                print(f"⚠️ Skipping {fpath}: {e}")
                continue

            save_dir = os.path.join(out_dir, str(year), chapter)
            os.makedirs(save_dir, exist_ok=True)

            serial = 1
            d2t.process(fpath,save_dir)
            # Loop tables to find those with "תרשים"
            for table in doc.tables:
                graph_name = None
                if len(table.rows) > 0:
                    # Concatenate all first-row cells for the name
                    graph_name = " ".join([cell.text.strip() for cell in table.rows[0].cells if cell.text.strip()])
                    if "תרשים" not in graph_name:
                        continue  # skip tables that aren’t graphs

                saved_blobs = set()  # to avoid duplicates

                # # Loop through all relationships in the DOCX
                # for rel_id, rel in doc.part._rels.items():
                #     target_ref = rel.target_ref.lower()

                #     # Normal images
                #     if "image" in target_ref:
                #         img_data = rel.target_part.blob
                #         if graph_name in saved_blobs:
                #             continue  # skip duplicates
                #         saved_blobs.add(graph_name)

                #         ext = rel.target_part.content_type.split("/")[-1]
                #         print(ext)
                #         if ext == "jpeg":
                #             ext = "jpg"

                #         filename = f"{year}_{chapter}_{serial}.{ext}"
                #         path = os.path.join(save_dir, filename)
                #         with open(path, "wb") as f:
                #             f.write(img_data)

                #         graph_summary[f"{year}_{chapter}_{serial}"] = graph_name or None
                #         serial += 1


                #     # Embedded Excel objects (OLE)
                #     elif "oleobject" in target_ref or "package" in target_ref:
                #         ole_data = rel.target_part.blob
                #         if ole_data in saved_blobs:
                #             continue  # skip duplicates
                #         saved_blobs.add(ole_data)

                #         if is_excel_ole(ole_data):  # your function to detect Excel OLE
                #             filename = f"{year}_{chapter}_{serial}.xlsx"
                #             path = os.path.join(save_dir, filename)
                #             with open(path, "wb") as f:
                #                 f.write(ole_data)

                #             graph_summary[f"{year}_{chapter}_{serial}"] = graph_name or None
                #             serial += 1

    # Save JSON mapping
    with open(os.path.join(out_dir, "graphs_summary.json"), "w", encoding="utf-8") as f:
        json.dump(graph_summary, f, ensure_ascii=False, indent=2)

    print("✅ Extraction finished")

# Run extraction
extract_graphs_from_reports()

📂 Year 2001
✅ Extraction finished


In [22]:
from docx import Document
import os

docx_path = "/content/reports/2001/02_02.docx"
output_dir = "images_from_docx"
os.makedirs(output_dir, exist_ok=True)

doc = Document(docx_path)

count = 0
for rel in doc.part._rels.values():
    if "image" in rel.target_ref:
        img_data = rel.target_part.blob
        img_ext = os.path.splitext(rel.target_ref)[1]
        count += 1
        with open(os.path.join(output_dir, f"image_{count}{img_ext}"), "wb") as f:
            f.write(img_data)

print(f"Extracted {count} images to '{output_dir}'")

!apt-get install -y imagemagick
!pip install Wand

from wand.image import Image
import os

wmf_path = "images_from_docx/image_2.wmf"
png_path = "images_from_docx/image_2.png"

with Image(filename=wmf_path) as img:
    img.format = 'png'
    img.save(filename=png_path)

print("Converted WMF to PNG!")



Extracted 7 images to 'images_from_docx'
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Converted WMF to PNG!


In [26]:
import os
from docx import Document
from wand.image import Image

##extracting images by file name format is ok with 2001-2016
def extract_and_convert_images(docx_path, output_dir, base_name="image"):
    """
    Extracts all images from a DOCX file and converts WMF to PNG.

    Parameters:
        docx_path (str): Path to the DOCX file.
        output_dir (str): Directory to save extracted images.
        base_name (str): Base name for saved images.
    """
    os.makedirs(output_dir, exist_ok=True)
    doc = Document(docx_path)
    count = 0

    for rel in doc.part._rels.values():
        if "image" in rel.target_ref:
            img_data = rel.target_part.blob
            img_ext = os.path.splitext(rel.target_ref)[1].lower()
            count += 1
            img_filename = os.path.join(output_dir, f"{base_name}_{count}{img_ext}")

            # Save the original image first
            with open(img_filename, "wb") as f:
                f.write(img_data)

            # Convert WMF to PNG
            if img_ext == ".wmf":
                png_filename = os.path.join(output_dir, f"{base_name}_{count}.png")
                with Image(filename=img_filename) as img:
                    img.format = 'png'
                    img.save(filename=png_filename)
                os.remove(img_filename)  # optional: remove original WMF
                print(f"Converted WMF to PNG: {png_filename}")
            else:
                print(f"Saved image: {img_filename}")

    print(f"Done! Extracted {count} images to '{output_dir}'.")

# Example usage
!apt-get install -y imagemagick
!pip install Wand

docx_file = "/content/reports/2016/02_02.docx"
save_dir = "/content/images_from_docx2"
extract_and_convert_images(docx_file, save_dir, base_name="2001_02_")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Saved image: /content/images_from_docx2/2001_02__1.jpeg
Saved image: /content/images_from_docx2/2001_02__2.jpeg
Saved image: /content/images_from_docx2/2001_02__3.jpeg
Saved image: /content/images_from_docx2/2001_02__4.jpeg
Saved image: /content/images_from_docx2/2001_02__5.jpeg
Saved image: /content/images_from_docx2/2001_02__6.jpeg
Saved image: /content/images_from_docx2/2001_02__7.jpeg
Saved image: /content/images_from_docx2/2001_02__8.jpeg
Saved image: /content/images_from_docx2/2001_02__9.jpeg
Saved image: /content/images_from_docx2/2001_02__10.jpeg
Saved image: /content/images_from_docx2/2001_02__11.jpeg
Saved image: /content/images_from_docx2/2001_02__12.jpeg
Done! Extracted 12 images to '/content/images_from_docx2'.


In [56]:
from docx import Document
import os
from wand.image import Image
from docx import Document
import os
from wand.image import Image

graph_summary = {}
def extract_images_from_docx_tables(docx_path, output_dir, chapter, year, base_name="image"):
    os.makedirs(output_dir, exist_ok=True)
    doc = Document(docx_path)
    count = 0

    def save_image(img_data, img_ext, idx):
        img_filename = os.path.join(output_dir, f"{base_name}_{idx}{img_ext}")
        with open(img_filename, "wb") as f:
            f.write(img_data)

        # Convert WMF to PNG
        if img_ext == ".wmf":
            png_filename = os.path.join(output_dir, f"{base_name}_{idx}.png")
            with Image(filename=img_filename) as img:
                img.format = 'png'
                img.save(filename=png_filename)
            os.remove(img_filename)
            return png_filename
        return img_filename

    # Iterate over tables
    for table in doc.tables:
        # Get full text of the first row (all cells combined)
        first_row_text = " | ".join(cell.text.strip() for cell in table.rows[0].cells)
        if not "תרשים" in first_row_text:
           continue
        found_image = False
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    text_before = para.text.strip()
                    for run in para.runs:
                        # Look for inline images
                        for blip in run._element.xpath(".//a:blip"):
                            rId = blip.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                            img_part = doc.part.related_parts[rId]
                            img_data = img_part.blob
                            img_ext = os.path.splitext(img_part.partname)[1].lower()
                            count += 1
                            img_filename = save_image(img_data, img_ext, count)
                            ##print(f"Saved image: {img_filename}, text before: '{text_before}'")
                            found_image = True
                            graph_summary[f'{count}_{chapter}_{year}']= first_row_text
                            break
                        if found_image:
                            break
                    if found_image:
                        break
                if found_image:
                    break
            if found_image:
                break


base_dir = "/content/reports"
save_dir = "/content/graphs_2001-2016"

for year in range(2001, 2016):  # 2001–2016 inclusive
    year_folder = os.path.join(base_dir, str(year))

    for chapter in range(1, 17):  # 01–16
        chapter_str = f"{chapter:02}"

        # Construct filename like "02_02.docx"
        docx_file = os.path.join(year_folder, f"{chapter_str}_{chapter_str}.docx")
        if not os.path.exists(docx_file):
            continue  # skip missing files

        # Output directory for this chapter
        out_path = os.path.join(save_dir, str(year), chapter_str)
        os.makedirs(out_path, exist_ok=True)

        # Base name for images, e.g. "2001_02_"
        base_name = f"{year}_{chapter_str}_"

        extract_images_from_docx_tables(docx_file, out_path,chapter,year, base_name=base_name)
        print("finish extracting for year: ", year, "chapter: ", chapter)

# Save JSON mapping
with open(os.path.join("/content/graphs_2001-2017", "graphs_summary.json"), "w", encoding="utf-8") as f:
    json.dump(graph_summary, f, ensure_ascii=False, indent=2)
print(f"Done! Extracted {count} images to '{output_dir}'.")


finish extracting for year:  2001 chapter:  1


KeyboardInterrupt: 

In [31]:
for year in range(2001, 2003):  # change to desired range
    print(f"📂 Year {year}")
    year_path = os.path.join(base_dir, str(year))
    if not os.path.isdir(year_path):
        continue

    for fname in os.listdir(year_path):
        if not fname.endswith(".docx"):
            continue

        chapter = fname.split("_")[0]
        fpath = os.path.join(year_path, fname)

        try:
            doc = Document(fpath)
        except Exception as e:
            print(f"⚠️ Skipping {fpath}: {e}")
            continue

        save_dir = os.path.join(out_dir, str(year), chapter)
        os.makedirs(save_dir, exist_ok=True)

        # Construct base_name like in your example: "2006_02_"
        base_name = f"{year}_{chapter}_"
        extract_and_convert_images(fpath, save_dir, base_name=base_name)

📂 Year 2001
Converted WMF to PNG: /content/graphs/2001/06/2001_06__1.png
Converted WMF to PNG: /content/graphs/2001/06/2001_06__2.png
Converted WMF to PNG: /content/graphs/2001/06/2001_06__3.png
Converted WMF to PNG: /content/graphs/2001/06/2001_06__4.png
Converted WMF to PNG: /content/graphs/2001/06/2001_06__5.png
Done! Extracted 5 images to '/content/graphs/2001/06'.
Converted WMF to PNG: /content/graphs/2001/03/2001_03__1.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__2.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__3.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__4.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__5.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__6.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__7.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__8.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__9.png
Converted WMF to PNG: /content/graphs/2001/03/2001_03__10.png


KeyboardInterrupt: 